In [1]:
!git clone https://github.com/hide-ous/semeval26_starter_pack.git

fatal: destination path 'semeval26_starter_pack' already exists and is not an empty directory.


In [2]:
import json
import pandas as pd

DATA_PATH = "/content/train_redacted.jsonl"

def load_psycomark(path=DATA_PATH):
    rows = []
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            rows.append(json.loads(line))
    return pd.DataFrame(rows)

df = load_psycomark()
df.head()

,_id,conspiracy,markers,subreddit,annotator
0,t1_f7ju17o,Yes,"[{'startIndex': 8, 'endIndex': 15, 'type': 'Ev...",conspiracy,annotator_0
1,t1_k5c5yyz,Can't tell,[],HighStrangeness,annotator_1
2,t1_givys64,No,"[{'startIndex': 0, 'endIndex': 7, 'type': 'Act...",europe,annotator_2
3,t1_joq538t,Can't tell,"[{'startIndex': 0, 'endIndex': 9, 'type': 'Act...",Republican,annotator_3
4,t1_hlk1vci,No,"[{'startIndex': 96, 'endIndex': 114, 'type': '...",ForwardPartyUSA,annotator_4


In [3]:
%cd /content/semeval26_starter_pack
!ls

/content/semeval26_starter_pack
datasheet.md	      infer_one_span.py      submission.zip
dev_redacted.jsonl    LICENSE		     train_binary.py
dev_rehydrated.jsonl  README.md		     train_one_span.py
eval_binary.py	      rehydrate_data.py      train_redacted.jsonl
eval_token.py	      requirements.txt	     train_rehydrated.jsonl
infer_binary.py       submission_example.py


In [4]:
!cp /content/train_redacted.jsonl /content/semeval26_starter_pack/train_redacted.jsonl

In [5]:
%cd /content/semeval26_starter_pack
!ls

/content/semeval26_starter_pack
datasheet.md	      infer_one_span.py      submission.zip
dev_redacted.jsonl    LICENSE		     train_binary.py
dev_rehydrated.jsonl  README.md		     train_one_span.py
eval_binary.py	      rehydrate_data.py      train_redacted.jsonl
eval_token.py	      requirements.txt	     train_rehydrated.jsonl
infer_binary.py       submission_example.py


In [ ]:
!python rehydrate_data.py
!ls

/content/semeval26_starter_pack/rehydrate_data.py:29: SyntaxWarning: invalid escape sequence '\W'
  return re.sub("http(.+)?(\W|$)", url_replacement_token, x)
Rehydrating comments:   0% 0/9 [00:00<?, ?it/s]Error during API request: 520 Server Error: <none> for url: https://arctic-shift.photon-reddit.com/api/comments/ids?ids=f7ju17o%2Ck5c5yyz%2Cgivys64%2Cjoq538t%2Chlk1vci%2Ccguw8w8%2Cfmcunuq%2Cjolyktt%2Cfegajpj%2Ck4a1rz6%2Cexd9ccm%2Ckb55gnn%2Chsoj4b0%2Ce9gbebp%2Cil8cpvx%2Cgzs5v7g%2Cdxeq6v9%2Chaja09s%2Cj0cxfzk%2Cdwjjh4l%2Cfnua8i5%2Cgteuknx%2Chkwzb2z%2Chn2tsej%2Cho7i6ht%2Cgsdgupf%2Cerrzsjm%2Ceoryiia%2Cdw850o1%2Ci46k0jt%2Cgv484ay%2Cfgbnl96%2Celc14lm%2Ce4hcygt%2Cf17mla9%2Cfxi59ad%2Cdzqz120%2Cewibxb5%2Cdt5kqwa%2Chqgw7x2%2Cih5cggk%2Ch7hwebn%2Cfrfd8jq%2Cgruwcg4%2Cg4pcbzj%2Cif6rhqz%2Cguf2o95%2Cfoozu8o%2Chbkbe8c%2Cfkc2fl3%2Cj4rr72m%2Cettkcu3%2Chsxp5n6%2Cgt2ixvs%2Cdye3vh7%2Ce93zmh3%2Cgv0dchj%2Cita35vm%2Ce713bqm%2Cfj010mi%2Cejcjqoo%2Ck4fu291%2Cf7xdoof%2Chaw5wob%2Cjafdooe%2Cjlyhvz7%2Cgrdh47x%2Cdxfd

In [ ]:
import pandas as pd

df = pd.read_json(
    "/content/semeval26_starter_pack/train_rehydrated.jsonl",
    lines=True
)

df.head()

In [ ]:
import json

with open('/content/semeval26_starter_pack/train_rehydrated.jsonl', 'r') as f:
    first_line = json.loads(f.readline())

print(json.dumps(first_line, indent=2))

In [ ]:
!pip install -q spacy
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

In [ ]:
def clean_text(text: str) -> str:
    text = re.sub(r"https?://\S+|www\.\S+", " <URL> ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_row(text: str):
    cleaned = clean_text(text)
    doc = nlp(cleaned)
    sentences = [sent.text for sent in doc.sents]
    return doc, sentences

docs = []
sents = []

for t in df["text"]:
    doc, sent_list = preprocess_row(t)
    docs.append(doc)
    sents.append(sent_list)

df["spacy_doc"] = docs
df["sentences"] = sents

df[["text", "sentences"]].head(3)


In [ ]:
MODAL_LEMMAS = {"must", "should", "could", "might", "may", "would", "will", "can"}
GENERIC_AGENTS = {"elite", "elites", "government", "they", "them", "globalists", "cabal"}

def extract_candidates(doc):
    candidates = []
    for i, sent in enumerate(doc.sents):
        text = sent.text.strip()
        lemmas = [t.lemma_.lower() for t in sent]
        words_lower = [t.text.lower() for t in sent]

        has_modal = any(l in MODAL_LEMMAS for l in lemmas)
        is_question = text.endswith("?")
        has_agent = any(w in GENERIC_AGENTS for w in words_lower)

        if has_modal or is_question or has_agent:
            candidates.append({
                "sent_index": i,
                "segment": text,
                "has_modal": has_modal,
                "is_question": is_question,
                "has_agent": has_agent,
            })
    return candidates

df["candidate_segments"] = df["spacy_doc"].apply(extract_candidates)

df["candidate_segments"].iloc[0]

cand_rows = []
for _, row in df.iterrows():
    for c in row["candidate_segments"]:
        cand_rows.append({
            "_id": row["_id"],
            "conspiracy": row["conspiracy"],
            **c,
        })

candidates_df = pd.DataFrame(cand_rows)
candidates_df.head()


In [ ]:
import json
import re
from collections import defaultdict, Counter
import pandas as pd

DATA_PATH = "/content/semeval26_starter_pack/train_rehydrated.jsonl"

token_cat_counts = defaultdict(Counter)

with open(DATA_PATH, "r", encoding="utf8") as f:
    for line in f:
        obj = json.loads(line)
        for m in obj.get("markers", []):
            mtext = m.get("text", "")
            mtype = m.get("type", None)
            if not mtype:
                continue

            tokens = [tok.lower() for tok in re.findall(r"[A-Za-z']+", mtext)]
            for tok in tokens:
                if tok:
                    token_cat_counts[tok][mtype] += 1

len(token_cat_counts)


In [ ]:
from collections import Counter

rows = []
for token, counter in token_cat_counts.items():
    total = sum(counter.values())
    dominant_type, dom_count = counter.most_common(1)[0]
    dominant_ratio = dom_count / total

    row = {
        "word": token,
        "total_count": total,
        "dominant_category": dominant_type,
        "dominant_ratio": dominant_ratio,
    }
    for cat in ["Actor", "Action", "Victim", "Evidence", "Threat", "Effect"]:
        row[f"{cat}_count"] = counter.get(cat, 0)
    rows.append(row)

dict_df = pd.DataFrame(rows)
dict_df.sort_values("total_count", ascending=False).head()

In [ ]:
stop_like = {
    "the","a","an","of","to","and","in","on","for","is","are","was","were","be","been","being",
    "this","that","these","those","it","its","i","you","he","she","we","they","them","my","your","our",
    "at","as","by","or","if","but","from","with","so","do","does","did","have","has","had","will","would",
    "can","could","should","may","might","about","just","not","no","yes","also","there","their","than",
    "then","when","what","which","who","how","why","because","into","out","up","down","over","under"
}

filt_df = dict_df[
    (dict_df["total_count"] >= 3) &
    (dict_df["dominant_ratio"] >= 0.6) &
    (~dict_df["word"].isin(stop_like))
].copy()

filt_df.sort_values(["dominant_category","total_count"], ascending=[True, False]).head(20)

In [ ]:
psy_dict = filt_df[["word", "dominant_category"]].rename(
    columns={"dominant_category": "category"}
)

psy_dict_path = "/content/psy_dict.csv"
psy_dict.to_csv(psy_dict_path, index=False)

psy_dict.head(), psy_dict_path

In [ ]:
import csv
from collections import defaultdict, Counter

DICT_PATH = "/content/psy_dict.csv"

def load_psy_dict(path=DICT_PATH):
    cat2words = defaultdict(set)
    with open(path, encoding="utf8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            cat2words[row["category"]].add(row["word"].lower())
    return dict(cat2words)

cat2words = load_psy_dict()
cat2words

In [ ]:
from collections import Counter

def liwc_features_for_doc(doc, cat2words):
    tokens = [t.text.lower() for t in doc if not (t.is_punct or t.is_space)]
    total = len(tokens) or 1
    counter = Counter(tokens)

    feats = {}
    for cat, vocab in cat2words.items():
        count = sum(counter[w] for w in vocab)
        feats[f"{cat}_count"] = count
        feats[f"{cat}_ratio"] = count / total
    return feats

df["liwc_doc_feats"] = df["spacy_doc"].apply(lambda d: liwc_features_for_doc(d, cat2words))

liwc_doc_df = pd.json_normalize(df["liwc_doc_feats"])
doc_features_df = pd.concat([df[["_id", "conspiracy"]], liwc_doc_df], axis=1)
doc_features_df.head()

In [ ]:
def liwc_features_for_segment(text, cat2words):
    doc = nlp(clean_text(text))
    tokens = [t.text.lower() for t in doc if not (t.is_punct or t.is_space)]
    total = len(tokens) or 1
    counter = Counter(tokens)

    feats = {}
    for cat, vocab in cat2words.items():
        count = sum(counter[w] for w in vocab)
        feats[f"{cat}_count"] = count
        feats[f"{cat}_ratio"] = count / total
    return feats

seg_rows = []
for _, row in candidates_df.iterrows():
    feats = liwc_features_for_segment(row["segment"], cat2words)
    seg_rows.append({**row, **feats})

segment_features_df = pd.DataFrame(seg_rows)
segment_features_df.head()

In [ ]:
import json

cand_out = "/content/candidate_segments_for_annotation.jsonl"

with open(cand_out, "w", encoding="utf8") as f:
    for _, row in candidates_df.iterrows():
        rec = {
            "id": f"{row['_id']}_{row['sent_index']}",
            "text": row["segment"],
            "meta": {
                "doc_id": row["_id"],
                "conspiracy": row["conspiracy"],
                "has_modal": row["has_modal"],
                "is_question": row["is_question"],
                "has_agent": row["has_agent"],
            }
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

cand_out

In [ ]:
doc_out = "/content/documents_for_annotation.jsonl"

with open(doc_out, "w", encoding="utf8") as f:
    for _, row in df.iterrows():
        rec = {
            "id": row["_id"],
            "text": row["text"],
            "meta": {
                "subreddit": row.get("subreddit", None),
                "existing_conspiracy_label": row["conspiracy"],
            }
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

doc_out

In [ ]:
df[["_id", "annotator", "conspiracy"]].head()

In [ ]:
from sklearn.metrics import cohen_kappa_score

pair_rows = []
for _id, group in df.groupby("_id"):
    if len(group["annotator"].unique()) == 2:
        pair_rows.append(group[["annotator", "conspiracy"]])

len(pair_rows)

In [ ]:
a1_labels = []
a2_labels = []

for g in pair_rows:
    g_sorted = g.sort_values("annotator")
    labels = list(g_sorted["conspiracy"])
    if len(labels) == 2:
        a1_labels.append(labels[0])
        a2_labels.append(labels[1])

cohen_kappa_score(a1_labels, a2_labels)

In [ ]:
from collections import defaultdict
import numpy as np

def spans_to_charset(markers, label_type):
    chars = set()
    for m in markers:
        if m["type"] == label_type:
            start = m["startIndex"]
            end = m["endIndex"]
            chars.update(range(start, end + 1))
    return chars

def f1_for_sets(a, b):
    if not a and not b:
        return 1.0
    inter = len(a & b)
    if inter == 0:
        return 0.0
    prec = inter / (len(a) or 1)
    rec = inter / (len(b) or 1)
    if prec + rec == 0:
        return 0.0
    return 2 * prec * rec / (prec + rec)

In [ ]:
label_types = ["Actor", "Action", "Evidence", "Victim", "Threat", "Effect"]

type_scores = {lt: [] for lt in label_types}

for _id, group in df.groupby("_id"):
    ann_ids = group["annotator"].unique()
    if len(ann_ids) != 2:
        continue
    g = group.set_index("annotator")
    ann1, ann2 = ann_ids[0], ann_ids[1]
    m1 = g.loc[ann1, "markers"]
    m2 = g.loc[ann2, "markers"]

    for lt in label_types:
        set1 = spans_to_charset(m1, lt)
        set2 = spans_to_charset(m2, lt)
        score = f1_for_sets(set1, set2)
        type_scores[lt].append(score)

span_iaa = {lt: float(np.mean(scores)) if scores else None
            for lt, scores in type_scores.items()}

span_iaa